In [ ]:
#GPT revised

import pandas as pd

# 定义一个函数来安全地解析每一行
def safe_parse_line(line):
    parts = line.split('|')
    if len(parts) == 6:  # 检查行是否正确地分成了6部分
        return parts
    else:
        # 如果行的格式不正确，可以选择跳过或处理它
        return None

# 逐行读取文件并手动处理
rows = []
with open('tat_qa.csv', 'r', encoding='utf-8') as file:
    for line in file:
        parsed_line = safe_parse_line(line.strip())
        if parsed_line:
            rows.append(parsed_line)

# 将处理后的行转换为 DataFrame
df_split = pd.DataFrame(rows, columns=['doc_name', 'q_uid', 'question', 'answer', 'answer_scale', 'answer_type'])

# 保存为新的 CSV 文件
df_split.to_csv('tat_qa_split.csv', index=False)

# 打印结果以验证
print(df_split.head())

# 打印总行数以验证
print(f"Total rows: {df_split.shape[0]}")


In [ ]:
grouped_df = df_split.groupby('doc_name').size().reset_index(name='count')
grouped_df = grouped_df.sort_values(by='count', ascending=False)
import os
folder_path = './tat_docs'  # 請替換成你實際的資料夾路徑

def get_file_size(doc_name):
    file_path = os.path.join(folder_path, doc_name+".pdf")
    if os.path.exists(file_path):
        return os.path.getsize(file_path)
    return None

# 新增檔案大小欄位
grouped_df['file_size'] = grouped_df['doc_name'].apply(get_file_size)

# 計算檔案大小除以 count 的比值，一個count要花多少大小才能夠查詢到，越小越好
grouped_df['size_per_count'] = grouped_df['file_size'] / grouped_df['count'] 

# # 刪除包含 None 的行 (也就是含有 'pdf' 的檔案名)
grouped_df = grouped_df.dropna(subset=['file_size'])

# 按 count 列進行降序排序
grouped_df = grouped_df.sort_values(by='size_per_count', ascending=True)

grouped_df

In [ ]:
# 選取前10大的檔案
top_10 = grouped_df.head(10)

# 計算 count 總數加總
total_count_sum = top_10['count'].sum()

# 找出包含 'pdf' 的文件名列表
pdf_list = top_10['doc_name'].tolist()

# 輸出結果
print("前10大的檔案 count 總數加總:", total_count_sum)
print("PDF 文件列表:", pdf_list)


In [ ]:
!pip install requirements.txt
!export PYTHONPATH=./

In [ ]:
#修改paddle的原始碼，把tools前面增加paddle.
from run_parse import run_parse
import time
import json
results = {}
#開始時間:11:30 ，開始價格：未知 (first doc)
#start:1.22 已經花了0.35元 暫停，環境檔案位置有問題，已修正
#start:1.35

api_key = "YOURKEY"
# gaslog-ltd_2019有一些表格沒有取代掉（把原本的md轉成html），之後再重新處裡．
pdfs = ['stmicroelectronics_2019', 'teradyne-inc_2019', 'vmware-inc_2019', 'leidos-holdings_2019', 'finjan-holding-inc_2019', 'cornerstone-ondemand-inc_2019', 'mitek-systems_2019', 'oracle-corporation_2019', 'plexus-corp_2019']
for i in pdfs:
    start_time = time.time()
    name = "tat_docs/" + i + ".pdf"
    print(name)
    path, image_dict, GPT_CALL_COUNT, NOTSAMELENGTH = run_parse(name, api_key, finance=True, page=False)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    results[i] = {
        "path": path,
        "image_dict": image_dict,
        "time_elapsed": elapsed_time,
        "GPT_CALL_COUNT": GPT_CALL_COUNT,
        "NOTSAMELENGTH" : NOTSAMELENGTH,
    }
    
    print(results[i])

    # 生成文件名，包含当前时间戳
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"{i}_{timestamp}.json"
    
    # 将当前的结果保存为JSON文件
    json_data = json.dumps(results[i], indent=4)
    with open(filename, "w") as json_file:
        json_file.write(json_data)
